## List and the standard library (advanced)

We'll cover the following topics in this notebook:
- Warm-up: `filter`, `iter`
- Index-aware functions: `mapi`, `filteri`, `iteri`
- General iterators: `fold_left`, `fold_right`
- Exercises: using fold

Today, we begin with a few more complicated operations from [Lists](https://ocaml.org/manual/5.2/api/List.html) that we did not cover last time.

To warm-up, we start with two simple functions, `filter` and `iter`.

> `filter : ('a -> bool) -> 'a list -> 'a list`
>
> &emsp; `filter f l` returns all the elements of the list `l` that satisfy the predicate `f`. The order of the elements in the input list is preserved.

val length : 'a list -> int = <fun>


OCaml has a `unit` type with only one value: `()`. It is used as the return value for a computation that does not typically returns values, for example, the printing function `print_endline`. If `e` has type `unit`, then it could be sequenced by another computation (of any type) with a semicolon: `e; e'`.

Next, we look at `iter`, the way to apply a unit-valued function through a list. Is our implementation tail-recursive?

> `iter : ('a -> unit) -> 'a list -> unit`
>
> &emsp; `iter f [a1; ...; an]` applies function `f` in turn to `[a1; ...; an]`. It is equivalent to `f a1; f a2; ...; f an`.

Now we look at `mapi`, a more advanced version of `map` such that `f` has access to the element's current position.

> `mapi : (int -> 'a -> 'b) -> 'a list -> 'b list`
>
> &emsp; Same as List.map, but the function is applied to the index of the element as first argument (counting from 0), and the element itself as second argument.

Like `map` and `mapi`, we have `filter` and `filteri` for the slightly more complicated situations.

> `filteri : (int -> 'a -> bool) -> 'a list -> 'a list`
>
> &emsp; Same as `List.filter`, but the predicate is applied to the index of the element as first argument (counting from 0), and the element itself as second argument.

For example, if we want to remove all elements in the list that at positions `3, 6, 9, ...`:

In [ ]:
let filter_3m xs = filteri (fun i x -> i mod 3 <> 0) xs

In [ ]:
filter_3m [0; 1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12]

Similarly, we also have `iteri` whose function argument has access to the current position.

> `val iteri : (int -> 'a -> unit) -> 'a list -> unit`
>
> Same as `List.iter`, but the function is applied to the index of the element as first argument (counting from `0`), and the element itself as second argument.

Given a list of integers, we can compute the `sum` and `product` of the list, using tail recursion.

We can observe a pattern here: both `sum'` and `product'`
  - carries an accumulator
  - returns the accumulator at the base case
  - uses some operation to combine the accumulator and the head element, then go through the tail recursively at the inductive case

In general, we can abstract ths pattern out, and define an operation called fold (order of evaluation is left to right).

> `fold_left : ('acc -> 'a -> 'acc) -> 'acc -> 'a list -> 'acc`
> 
> &emsp; `fold_left f init [b1; ...; bn]` is `f (... (f (f init b1) b2) ...) bn`.

We get `sum` by setting `f = (+)` and `init = 0`, and `product` by setting `f = (*)` and `init = 1`.

We may also fold with order from right to left.

> `fold_right : ('a -> 'acc -> 'acc) -> 'a list -> 'acc -> 'acc`
>
> &emsp; `fold_right f [a1; ...; an] init` is `f a1 (f a2 (... (f an init) ...))`. Not tail-recursive.

Now, let us solve a few exercise questions, directly and using the library functions we learned today and yesterday.

1. (a) Write a function `product` so that `product [a; b; c; d]`
   computes the product `a * b * c * d`.

   (b) Write a tail-recursive version of `product`.
   
   (c) What does `product [-1;-2;0;1;2;3]` return?  Is your
   implementation as efficient as possible?


Part (a) is simply an instance of the fold function, but let's complete it in the straightforward way, in order to do part (b).

Part (b) is done via an accumulator.

For part (c), we know that everything times zero is zero, so the function should immediately return zero when it sees a zero in the list, to have better time efficiency. Note that we could use OCaml's pattern matching to do the comparison for us.

2. Write a function `count` so that `count x xs` returns the number of
   times `x` occurs in `xs`.

We start with the simplest way of implementing it.

It is not hard to see that this function could be easily turned into a tail-recursive one, using the accumulator to store the count.


3. Write a function `repeat` so that `repeat n x` returns a list with
   `n` copies of `x`.  For example, `repeat 4 10` should return the
   list `[10; 10; 10; 10]`.

We get `repeat` by calling `List.init` with a constant function that always returns `x`.


4. Write a function `shorter` so that `shorter xs ys` returns the
   shorter of the two lists `xs` and `ys`.  Can you avoid examining
   every cons cell in each list?

To do this, we first define a function that compares the lengths of `xs` and `ys`. Note that this is provided in the standard library:
> `compare_lengths : 'a list -> 'b list -> int`
> 
> Compare the lengths of two lists. `compare_lengths l1 l2` is equivalent to `compare (length l1) (length l2)`, except that the computation stops after reaching the end of the shortest list.

5. Write a function `is_sorted` to detect if a list is already in sorted order.

This is another example of fancy pattern matching: we could match for a singleton list, or a list with at least two elements.